In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import math

from munch import Munch
from qsr_learning.data import DRLDataset
from qsr_learning.models import DRLNet
from torch.utils.data import DataLoader

config = Munch()

# Dataset

In [ ]:
config.dataset = Munch(
    entity_names=["octopus", "trophy"],
    relation_names=["left_of", "right_of"],
    num_entities=2,
    frame_of_reference="absolute",
    w_range=(32, 32),
    h_range=(32, 32),
    theta_range=(0, 2 * math.pi),
    filter_fn=None,
    add_bbox=False,
    add_front=False,
    transform=None,
    canvas_size=(224, 224),
    split=(8, 1, 1),
    total_size=2 ** 4,
    part="train",
)
dataset = DRLDataset(**config.dataset)

# Data Loader

In [ ]:
config.data_loader = Munch(
    batch_size=128,
    shuffle=True,
    num_workers=4,
)
data_loader = DataLoader(dataset, **config.data_loader)

# Model

In [ ]:
config.model = Munch(
    vision_model="resnet18",    
    image_size=(3, *config.dataset.canvas_size),
    num_embeddings=len(dataset.word2idx),    
    embedding_dim=10,
    question_len=dataset[0][1].shape.numel(),
)
model = DRLNet(**config.model)

for batch in data_loader:
    break

model(batch[0], batch[1])